## Dados apenas dos Estados Unidos

In [15]:
import pandas as pd
from Modulo_Senha import senha
import requests 
from selenium import webdriver
from selenium.webdriver.common.by import By

lista = pd.read_excel(r"C:\Users\evert\OneDrive\Documents\Base de dados\API\Lista_maior_bilheteria.xlsx")
display(lista)

,Rank,Title,Year
0,1,Avatar,2009
1,2,Avengers: Endgame,2019
2,3,Avatar: The Way of Water,2022
3,4,Titanic,1997
4,5,Star Wars: Episode VII - The Force Awakens,2015
...,...,...,...
195,196,Men in Black,1997
196,197,The Croods,2013
197,198,The Hangover Part II,2011
198,199,Iron Man,2008


In [16]:
# fazendo um único conjuto de dados de anos
anos = sorted(lista['Year'].unique())
anos = {'Year': anos}
tabela_inf = pd.DataFrame(anos)

In [17]:
# Calcular a inflação e colocar na coluna

navegador = webdriver.Chrome()
navegador.maximize_window() 


navegador.get("https://data.bls.gov/cgi-bin/cpicalc.pl")
valor = navegador.find_element(By.ID,"cost1").send_keys("1")
mes = navegador.find_element(By.ID,"year2-month").send_keys('January')
valores = []
for anos in tabela_inf:
    for ano in tabela_inf[anos]:
        ano = navegador.find_element(By.ID,"year1-year").send_keys(ano)
        procurar = navegador.find_element(By.ID,"submit").click()
        preco = navegador.find_element(By.ID,"answer").text
        valores.append(preco)
navegador.quit()

tabela_inf['Inflação'] = valores
                     

In [18]:
display(tabela_inf)

,Year,Inflação
0,1977,$5.27
1,1982,$3.27
2,1993,$2.16
3,1994,$2.11
4,1996,$2.00
5,1997,$1.94
6,1999,$1.88
7,2001,$1.76
8,2002,$1.74
9,2003,$1.70


In [19]:
# Adicionar os valores arrecadados dos filmes e após fazer a multiplicação da inflação para ajustar para o atual periodo

bilheteria = []
for nome in lista['Title']:
    link = f'http://www.omdbapi.com/?t={nome}&apikey={senha()}'
    requisicao = requests.get(link)
    informacoes = requisicao.json()
    bilheteria.append(informacoes['BoxOffice'])

lista['Bilheteria'] = bilheteria


In [139]:
nova = lista.merge(tabela_inf)
display(nova)

,Rank,Title,Year,Bilheteria,Inflação
0,1,Avatar,2009,"$785,221,649",$1.46
1,2,Avengers: Endgame,2019,"$858,373,000",$1.23
2,3,Avatar: The Way of Water,2022,"$684,075,767",$1.10
3,4,Titanic,1997,"$674,292,608",$1.94
4,5,Star Wars: Episode VII - The Force Awakens,2015,"$936,662,225",$1.32
...,...,...,...,...,...
195,196,Men in Black,1997,"$250,690,539",$1.94
196,197,The Croods,2013,"$187,168,425",$1.34
197,198,The Hangover Part II,2011,"$254,464,305",$1.40
198,199,Iron Man,2008,"$319,034,126",$1.46


In [152]:
# Fazendo tratamento na tabela

# Retirando o '$'
nova['Bilheteria_'] = nova['Bilheteria'].str.replace('$', '')
nova['Inflação_'] = nova['Inflação'].str.replace('$', '')

# Alterando o 'N/A' para '0'
nova['Bilheteria_'] = nova['Bilheteria_'].str.replace('N/A','0')
nova['Inflação_'] = nova['Inflação_'].str.replace('N/A','0')
nova['Bilheteria_'] = nova['Bilheteria_'].str.replace(' ', '')
nova['Inflação_'] = nova['Inflação_'].str.replace(' ', '')

# Modificando os tipos de valores das colunas
nova['Bilheteria_'] = nova['Bilheteria_'].str.replace(',', '').astype(int)
nova['Inflação_'] = nova['Inflação_'].str.replace(',', '').astype(float)


In [153]:
#Atualizando os valores de acordo com a inflação
nova['Total'] = nova['Bilheteria_'] * nova['Inflação_']

# Ordenando a tabela de acordo com o valor da inflação
tabela_ordenada = nova.sort_values(by='Total', ascending=False)

In [154]:
# Atualizando o Rank
novo_rank = []
for valor in range(1,201):
    novo_rank.append(valor)

tabela_ordenada['Rank_Novo'] = novo_rank
tabela_ordenada['saldo'] = tabela_ordenada['Rank'] - tabela_ordenada['Rank_Novo']
tabela_ordenada['saldo'] = [f"+{valor}" if valor > 0 else str(valor) for valor in tabela_ordenada['saldo']]
tabela_ordenada['Rank_Novo'] = tabela_ordenada['Rank_Novo'].astype(str)
tabela_ordenada['saldo'] = tabela_ordenada['saldo'].astype(str)
tabela_ordenada['Rank_Novo'] = tabela_ordenada['Rank_Novo'] + ' ' + '(' + tabela_ordenada['saldo'] + ')'

In [155]:
# Ordenando as colunas 
tabela_ordenada = tabela_ordenada[['Rank','Rank_Novo', 'Title', 'Year', 'Bilheteria', 'Inflação', 'Total']]


#Renomeando colunas:
renomeando = {'Rank': 'Rank','Rank_Novo': 'Rank_Novo', 'Title': 'Titulo', 'Year': 'Ano', 'Bilheteria': 'Bilheteria', 'Inflação': 'Inflação', 'Total': 'Total'}
tabela_ordenada = tabela_ordenada.rename(columns=renomeando)
display(tabela_ordenada)

,Rank,Rank_Novo,Titulo,Ano,Bilheteria,Inflação,Total
115,116,1 (+115),Star Wars: Episode IV - A New Hope,1977,"$460,998,507",$5.27,2.429462e+09
106,107,2 (+105),E.T. the Extra-Terrestrial,1982,"$439,454,989",$3.27,1.437018e+09
3,4,3 (+1),Titanic,1997,"$674,292,608",$1.94,1.308128e+09
4,5,4 (+1),Star Wars: Episode VII - The Force Awakens,2015,"$936,662,225",$1.32,1.236394e+09
0,1,5 (-4),Avatar,2009,"$785,221,649",$1.46,1.146424e+09
...,...,...,...,...,...,...,...
74,75,196 (-121),The Battle at Lake Changjin,2021,"$342,411",$1.18,4.040450e+05
176,177,197 (-20),Water Gate Bridge,2022,"$117,294",$1.10,1.290234e+05
156,157,198 (-41),Full River Red,2023,N/A,$1.03,0.000000e+00
151,152,199 (-47),Detective Chinatown 3,2021,N/A,$1.18,0.000000e+00


In [156]:
# Exportar a tabela
tabela_ordenada.to_excel('Maiores_Bilheterias_Atualizado.xlsx', index=False)